In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import json


with open('dataset/alqac25_private_test_task2_combined.json', 'r', encoding='utf-8') as f:
    private_test = json.load(f)

with open("dataset/alqac25_law.json", "r", encoding="utf-8") as f:
    laws = json.load(f)


article_mapping = {
    item['id']: {article["id"]: article for article in item["articles"]}
    for item in laws
}

def get_text(law_id, article_id):
    law_id = str(law_id)
    article_id = str(article_id)        
    text = article_mapping.get(law_id, {}).get(article_id, {}).get("text", "")
    if not text:
        raise ValueError(f"Article {article_id} of Law {law_id} not found.")
    return text

splitter = RecursiveCharacterTextSplitter(separators=[
                                          "\n\n", r"\n\d+\. Sửa đổi", "\n\d+", "\n", " ", ""], is_separator_regex=True, chunk_size=500, chunk_overlap=100)
split_texts = {}

for law_id, articles in article_mapping.items():
    for article_id in articles:
        if law_id not in split_texts:
            split_texts[law_id] = {}
        split_texts[law_id][article_id] = splitter.split_text(
            get_text(law_id, article_id))

def get_text(law_id, article_id):
    law_id = str(law_id)
    article_id = str(article_id)        
    text = article_mapping.get(law_id, {}).get(article_id, {}).get("text", "")
    if not text:
        raise ValueError(f"Article {article_id} of Law {law_id} not found.")
def process_relevant_articles(articles):
    results = []
    for item in articles:
        law_id = item['law_id']
        article_id = item['article_id']
        texts = split_texts[law_id][article_id]
        for idx, text in enumerate(texts):
            results.append({
                'law_id': law_id,
                'article_id': article_id,
                'text': text,
                'index': idx
            })
    return results


for item in private_test:
    item['relevant_articles'] = process_relevant_articles(
        item['relevant_articles'])

with open('dataset/alqac25_private_test_task2_combined_processed.json', 'w', encoding='utf-8') as f:
    json.dump(private_test, f, indent=4, ensure_ascii=False)